In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'usa-2016-presidential-election-by-county.csv',';')

In [12]:
df.shape

(3143, 160)

In [11]:
df.loc[df['State'] == 'North Carolina'].head()

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),...,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2,first_number
15,North Carolina,NC,37051,"Cumberland County, North Carolina",77.0,124402.0,74693.0,75792.0,52151.0,50666.0,...,127641.0,731.0,1183.0,0.009268,0.005730,0.400655,0.411143,0.599345,0.588857,1
16,North Carolina,NC,37167,"Stanly County, North Carolina",23.0,29451.0,8878.0,8431.0,19329.0,19904.0,...,28717.0,299.0,382.0,0.013302,0.010489,0.702453,0.685255,0.297547,0.314745,2
51,North Carolina,NC,37031,"Carteret County, North Carolina",28.0,36919.0,11130.0,10301.0,23131.0,24775.0,...,35517.0,336.0,441.0,0.012417,0.009712,0.706323,0.675141,0.293677,0.324859,3
83,North Carolina,NC,37195,"Wilson County, North Carolina",24.0,37593.0,19652.0,20875.0,17375.0,17954.0,...,39109.0,164.0,280.0,0.007159,0.004410,0.462386,0.469252,0.537614,0.530748,3
100,North Carolina,NC,37001,"Alamance County, North Carolina",37.0,69264.0,28918.0,28875.0,34859.0,38170.0,...,67776.0,576.0,731.0,0.010786,0.008951,0.569319,0.546576,0.430681,0.453424,6


In [4]:
df['first_number'] = df['Votes'].astype(str).str[:1].replace('n',np.nan)

In [5]:
df_minus_no_vote_counties = df.dropna(subset = ['first_number']) #Remove rows where there was no vote value

In [6]:
print(df.shape)
print(df.dropna(subset = ['first_number']).shape)

(3143, 160)
(3111, 160)


In [7]:
states = df['State'].unique()

In [8]:
benfords_percents = {
    1:.301,
    2:.176,
    3:.125,
    4:.097,
    5:.079,
    6:.067,
    7:.058,
    8:.052,
    9:.046,
}

In [9]:
number_map = {
    1:'One',
    2:'Two',
    3:'Three',
    4:'Four',
    5:'Five',
    6:'Six',
    7:'Seven',
    8:'Eight',
    9:'Nine',              
}

In [29]:
data_list = []
# ITERATE OVER EACH STATE
for state in states:
    this_state_df = df.loc[df['State'] == state]
    
    #GROUP BY FIRST NUMBER AND COUNT HOW MANY THERE ARE
    this_state_df_number_grouped = this_state_df.groupby('first_number')
    this_state_df_number_grouped.count()
    votes_df = this_state_df_number_grouped['Votes'].count().reset_index()
    if len(votes_df) > 0:
        data_dict = {
            'State': state,
            'Total': votes_df['Votes'].sum()
        } 

        diff_sum = 0
        abs_diff_sum = 0
        diff_count = 0
        
        # ITERATE OVER EACH NUMBER
        for number in range(1,10):        
            
            # CREATE COLUMN NAMES
            number_name = f'{number_map[number]} Count'
            number_percent_name = f'{number_name} Result %'
            number_expectation_name = f'{number_name} Expectation %'            
            number_difference_name = f'{number_name} Difference %'  
            
            # CALCULATE AND ASSIGN THE VALUE AND PERCENT TO COLUMNS
            value = votes_df['Votes'].loc[votes_df['first_number'] == str(number)]       
            
            # PROCEED IF THERE ARE ANY VALUES FOR THIS NUMBER
            if len(value) > 0:
                data_dict[number_name] = int(value)
                data_dict[number_percent_name] = round(data_dict[number_name]/data_dict['Total'] * 100,2)
                
                # CALCULATE AND ASSIGN EXPECTATION
                data_dict[number_expectation_name] = round(data_dict['Total'] * benfords_percents[number],2)

                delta = data_dict[number_percent_name] - data_dict[number_expectation_name]

                # CALCULATE AND ASSIGN DIFFERENCE BETWEEN ACTUAL AND EXPECTATION            
                if delta != 0:
                    data_dict[number_difference_name] = round((delta/data_dict[number_expectation_name]) * 100,2)  
                else:
                    data_dict[number_difference_name] = 0

                # TALLY ONGOING SUM AND COUNT TO BE USED FOR AGGREGATE VALUES
                diff_sum += data_dict[number_difference_name]
                abs_diff_sum += abs(data_dict[number_difference_name])
                diff_count +=1                                  
                
        # CALCULATE AND ASSIGN AGGREGATE VALUES
        data_dict['diff_count'] =  diff_count
        data_dict['diff_sum'] =  diff_sum

        if diff_count > 0:
            
            data_dict['Average Net Difference %'] =  round((diff_sum/diff_count),2)
            data_dict['Average Absolute Difference %'] =  round((abs_diff_sum/diff_count),2)
            abs_diff = data_dict['Average Absolute Difference %']
        else:
            abs_diff = 'NA'
        
        # APPEND TO LIST
        data_list.append(data_dict)
        print(f'{state}: {abs_diff}')
    else:
        print(f'{state}: No votes')
        
# CONVERT LIST TO DATAFRAME    
new_df = pd.DataFrame(data_list)
print('Median % Difference:', new_df['Average Absolute Difference %'].median())

Alaska: No votes
Nebraska: 41.08
California: 78.25
Florida: 67.27
Kansas: 24.18
Kentucky: 19.44
Michigan: 51.95
Missouri: 22.2
Illinois: 40.63
Virginia: 23.24
Oregon: 220.6
Minnesota: 33.94
North Carolina: 8.07
New Mexico: 210.29
Oklahoma: 39.69
Colorado: 53.46
Georgia: 32.07
Ohio: 31.76
Pennsylvania: 47.37
South Carolina: 102.19
Arkansas: 64.21
Utah: 257.13
Iowa: 41.59
Texas: 60.27
Maryland: 354.28
Tennessee: 30.75
Wisconsin: 66.41
New Jersey: 463.15
New York: 67.52
Alabama: 78.5
Idaho: 156.14
Indiana: 36.88
South Dakota: 49.17
Massachusetts: 1005.15
Washington: 159.94
Wyoming: 431.08
Louisiana: 62.77
Mississippi: 71.19
Montana: 128.96
Rhode Island: 5037.54
West Virginia: 95.75
New Hampshire: 1168.77
Maine: 648.87
Nevada: 523.33
North Dakota: 99.53
Connecticut: 2139.62
Delaware: 9765.96
Vermont: 882.19
Hawaii: 7971.54
Arizona: 972.18
District of Columbia: 55455.56
Median % Difference: 74.72


##### Last year this ^^ (Median % Difference) was the average discrepency between the totals of each state's first-number count and the expected count based on Benford's Law. 

##### This means that each number can be expected to be incorrect by +/- 78%. Which makes this a nearly useless way to detect anomolies since the results are largely anomolies.

In [31]:
new_df.head()

,Average Absolute Difference %,Average Net Difference %,Eight Count,Eight Count Difference %,Eight Count Expectation %,Eight Count Result %,Five Count,Five Count Difference %,Five Count Expectation %,Five Count Result %,...,Three Count Difference %,Three Count Expectation %,Three Count Result %,Total,Two Count,Two Count Difference %,Two Count Expectation %,Two Count Result %,diff_count,diff_sum
0,41.08,-3.80,1.0,-77.69,4.84,1.08,4.0,-41.50,7.35,4.30,...,85.11,11.62,21.51,93,20.0,31.40,16.37,21.51,9,-34.16
1,78.25,78.25,4.0,128.48,3.02,6.90,4.0,50.66,4.58,6.90,...,90.21,7.25,13.79,58,8.0,35.06,10.21,13.79,9,704.28
2,67.27,67.22,7.0,200.29,3.48,10.45,5.0,41.02,5.29,7.46,...,6.92,8.38,8.96,67,8.0,1.27,11.79,11.94,9,604.98
3,24.18,-14.66,6.0,4.58,5.46,5.71,5.0,-42.65,8.30,4.76,...,-5.64,13.12,12.38,105,24.0,23.70,18.48,22.86,9,-131.96
4,19.44,-7.89,7.0,-6.57,6.24,5.83,14.0,23.10,9.48,11.67,...,-33.33,15.00,10.00,120,19.0,-25.05,21.12,15.83,9,-71.00


In [32]:
new_df.to_csv('your_filename.csv')